In [2]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from scipy.signal import TransferFunction, lsim, cheby1, butter, bode, lti
from scipy.fft import fft, ifft

# Requirements
Interesting signal f_i =< 8 kHz, A_i =< 1
Noise signal f_n >= 11 kHz, A_n =< 1
No signal between 8 kHz to 11 kHz

# Task
Design a anti-aliasing filter (LP) followed by 12-bit AD converter sampling at f_s <= 24 kHz
Minimize order of anti-aliasing filter
Interference in final signal must not exceed 0.5/2^11 V. 
Maximum ripple 3 dB

# Thoughts
The minimal sampling rate f_s  > 2*beta, where beta is the maximum interesting frequency
Thus we need f_s > 16 kHz
Let's start with f_s = 17 kHz

Let's start by calculating the order of the Butterworth filter 
The order N is derived by: 
N = 1/2 * log(G_p / G_s) / log(w_p/w_s)
Where:
G_p is the gain of the pass-band: G_p = 1/(1-δ_p)^2 - 1
G_s is the gain of the stop-band: G_s = 1/δ_s^2 -1 
Note that δ_p is the maximum ripple/deviation in the pass-band and δ_s is the maximum ripple/deviation in the stop-band

w_p is the maximum frequency of the pass-band
w_s is the minimum frequency of the stop-band 
We choose w_p 8 kHz and w_s 11 kHz and caluclate the order:

In [3]:
w_p, w_s = 8000, 11000
# A_db = 20*log(A1/A2) -> A1 = A2*10^(A_db/20)
δ_p = 10**(3/20)  # 1.4125375446227544
δ_s = 0.5/2**11 # maximum interference allowed by specification
G_p = 1/(1-δ_p)**2 - 1 #4.875881669435278
G_s = 1/δ_s**2 -1  #

In [4]:
#Butter
N = 1/2 * np.log(G_p/G_s)/np.log(w_p/w_s)
print(N) # We need N >= 24

23.63173965273065


In [22]:
# plot the butterworth filter 
order = 24
b,a = butter(order, 8000*2*np.pi, 'lowpass', analog=True)
f_s = 17000
w, mag, phase = bode((b, a), w=np.logspace(0,6,2000))  # frequency response

plt.figure(figsize=(10, 6))
plt.semilogx(w / (2*np.pi), mag)  # convert rad/s → Hz
plt.title('Chebyshev Type I Lowpass Filter (analog)')
plt.ylabel('Magnitude [dB]')
plt.grid(True, which='both')
plt.show()

Let's also calculate the order needed for a Chebyshev I filter. They generally have a faster roll-off but might become unstable.
The order of the Chebyshev I filter is defined as:
N = cosh^-1 (sqrt(G_s/G_p))

The ripple control factor ϵ is defined as 

ϵ = sqrt(G_p) with G_p defined the same as above: G_p = 1/(1-δ_p)^2 - 1

In [6]:
# Chebyshev
w_p, w_s = 8000, 11000
# A_db = 20*log(A1/A2) -> A1 = A2*10^(A_db/20)
δ_p = 10**(3/20)  # 1.4125375446227544
δ_s = 0.5/2**11 # maximum interference allowed by specification
G_p = 1/(1-δ_p)**2 - 1 #4.875881669435278
G_s = 1/δ_s**2 -1  

ϵ = np.sqrt(1/(1-δ_p)**2 - 1) # 2.2081398663660954

N = np.arccosh(np.sqrt(G_s/G_p))/np.arccosh(w_s/w_p) # 9.772382819316107
print(N) 

9.772382819316107


We can clearly see the Chebyshev I filter is much lower order (10) than Butterworth (24)
We choose the Order 10 Chebyshev I filter

In [45]:
order = 10
b,a  = cheby1(order, 3, 8000*2*np.pi, 'lowpass', analog=True)
filter_c = lti(b,a)

In [44]:
f_s = 17000
w, mag, phase = bode((b, a), w=np.logspace(0,6,2000))  # frequency response

plt.figure(figsize=(10, 6))
plt.semilogx(w / (2*np.pi), mag)  # convert rad/s → Hz
plt.title('Chebyshev Type I Lowpass Filter (analog)')
plt.ylabel('Magnitude [dB]')
plt.grid(True, which='both')
plt.show()

In [75]:
# w = 2*pi*f
f_s = 17000
T_s = 1/f_s
n_samples = 10
t = np.arange(n_samples) * T_s

def x1(t):
    return np.sin(2*np.pi*8000*t) # 8 kHz

def x2(t):
    return np.sin(2*np.pi*12000*t) # 12 kHz


In [76]:
# Raw signals
plt.plot(t, x1(t), label="8 kHz signal")
plt.plot(t, x2(t), label="12 kHz signal")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude [V]")
plt.legend()
plt.show()

In [79]:
# Raw Superposition 
plt.plot(t, x1(t)+ x2(t), label="Superposition of signals")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude [V]")
plt.legend()
plt.show()

invalid command name "139643028895296process_stream_events"
    while executing
"139643028895296process_stream_events"
    ("after" script)
can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


In [ ]:
# Filtered Superposition  
fs_high = 10_000_000  # 10 MHz "analog simulation rate"
t_high = np.arange(0, 5e-3, 1/fs_high)  # 5 ms
x_super= x1(t_high) + x2(t_high)

t_out, y_out, _ = lsim(filter_c, x_super, t_high)
t_samples = np.arange(0,t_high[-1], T_s)
y_samples = np.interp(t_samples, t_out, y_out)

plt.plot(t_samples*1000, y_samples, label="Filtered & sampled superposition of signals")
#plt.plot(t_out*1000, y_out, label="Filtered superposition of signals")
plt.xlabel("Time [ms]")
plt.ylabel("Amplitude [V]")
plt.legend()
plt.show()

In [39]:
# Sampled signals
t1 = np.linspace(0, 15*T_s, 15) # 15 samples on a time of 15 sample periods -> sampling
x1_s = x1(t1)
x2_s = x2(t1)

plt.plot(t1, x1_s, label="1 kHz signal")
plt.plot(t1, x2_s, label="12 kHz signal")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude [V]")
plt.legend()
plt.show()